## Модульный подход к дизайну


## Модульный подход к дизайну
Ранее мы рассматривали высокоуровневые компоненты (абстрактные "квадраты"). Теперь углубимся в детали того, что происходит под капотом.

### Пример: Интернет-магазин
Вместо одного монолита "Магазин", мы выделяем подсервисы и сценарии:
1.  **Покупка**: Добавление в корзину -> Оформление.
2.  **Уведомления**: После покупки нужно отправить email/push.
3.  **Склад (Inventory)**: После покупки нужно уменьшить остаток товара.
4.  **Доставка**: Передача заказа в логистику.

**Акторы:**
*   **Покупатели**: Создают заказы.
*   **Продавцы**: Управляют товарами, смотрят статистику.
*   **Администраторы**: Модерация, поддержка.

### Преимущества и сложности
**Плюсы:**
*   Четкое разделение ответственности.
*   Независимое масштабирование (сервис уведомлений нагружен меньше, чем поиск товаров).
*   Легче переносить требования в код.

**Минусы:**
*   Сложность связей.
*   Зависимость сервисов друг от друга (если упал сервис товаров, корзина может не работать).

---

## Очереди сообщений (Message Queues)
Очередь сообщений позволяет разорвать жесткую связь между сервисами (Decoupling).

**Компоненты:**
*   **Producer (Продьюсер)**: Создает сообщение (например, "Заказ создан").
*   **Broker (Брокер)**: Хранит сообщение.
*   **Consumer (Консьюмер)**: Забирает сообщение и обрабатывает его (например, отправляет письмо).

**Сценарии:**
*   **Push-based**: Брокер сам толкает сообщение консьюмеру (низкая задержка).
*   **Pull-based**: Консьюмер сам приходит за сообщениями, когда готов (защита от перегрузки).

### RabbitMQ vs Apache Kafka

#### RabbitMQ
*   **Тип**: Традиционный Message Broker.
*   **Логика**: Умный брокер, глупый консьюмер.
*   **Маршрутизация (Exchanges)**:
    *   **Direct**: Точное совпадение ключа.
    *   **Fanout**: Всем подписчикам (Broadcasting).
    *   **Topic**: По шаблону (например, `logs.error.*`).
*   **Применение**: Сложная маршрутизация, задачи, которые нужно выполнить 1 раз (Background jobs).

#### Apache Kafka
*   **Тип**: Distributed Event Streaming Platform.
*   **Логика**: Глупый брокер, умный консьюмер (сам следит за Offset).
*   **Структура**:
    *   **Topic**: Поток данных (лог).
    *   **Partition**: Топик бьется на части для параллелизма.
    *   **Offset**: Позиция чтения.
*   **Хранение**: Хранит сообщения заданное время (Retention), даже после прочтения.
*   **Применение**: Логи, Стриминг данных, Event Sourcing, когда важна высокая пропускная способность.

---

## Практические примеры с очередями

### Сервис уведомлений
**Проблема**: Нельзя заставлять пользователя ждать, пока отправится SMS.
**Решение**:
1.  API принимает запрос "Отправить уведомление".
2.  Кладет задачу в RabbitMQ/Kafka.
3.  Сразу отвечает пользователю "ОК".
4.  Воркеры (Workers) в фоне разбирают очередь и шлют SMS.

### Сервис бронирования (Airbnb)
**Проблема**: Сложная цепочка действий после бронирования.
**Решение**:
1.  Пользователь нажал "Забронировать".
2.  Сервис пишет в БД "Pending" и кидает событие `BookingCreated`.
3.  Подписчики события:
    *   Сервис платежей -> Списывает деньги.
    *   Сервис календаря -> Блокирует даты.
    *   Сервис уведомлений -> Пишет владельцу жилья.
